Bringing the database as a pandas dataframe

In [8]:
import json
import pandas as pd

PATH_TO_MATERIALS  = 'materials.json'
with open (PATH_TO_MATERIALS) as f:
        d = json.load(f)
        # global material_df 
        material_df = pd.DataFrame.from_dict(d)

Showing the first 5 elements

In [19]:
material_df.head(5)

,Name (en),Name (de),Reference-size,Reference-unit,Density (kg/m3),Area weight (kg/m2),GWP,L1,L2,L3,L4,id
0,,Glas - Guardian Europe S.à r.l. - Beschichtete...,1.0,qm,2500.00,2.5,"{'A1': 2.78, 'A2': 0.000655, 'A3': 0.6, 'C1': ...",Components for windows and curtain walls,Infill,Transparent infill,None,0
1,,Glas - Guardian Europe S.à r.l. - Unbeschichte...,1.0,qm,2500.00,2.5,"{'A1': 0.66, 'A2': 0.0357, 'A3': 2.02, 'C1': 0...",Components for windows and curtain walls,Infill,Transparent infill,None,1
2,,Glas - Guardian Europe S.à r.l. - Verbundsiche...,1.0,qm,2500.00,2.5,"{'A1': 3.88, 'A2': 0.00451, 'A3': 0.59, 'C1': ...",Components for windows and curtain walls,Infill,Transparent infill,None,2
3,1.2.04Expanded clay sand,Blähton Sand,1.0,kg,NaN,NaN,{'A1-A3': 0.3772651625},Mineral building products,Concrete aggregates,Expanded clay,None,3
4,3- and 5-layer solid wood panel (German average),3- und 5-Schicht Massivholzplatte (Durchschnit...,1.0,m3,510.45,NaN,"{'A1': -763.8902040283, 'A2': 25.1403012207, '...",Wood,Derived timber products,3- and 5-ply wood,None,4


Experimenting with how to filter. Similar to a query, we can search for speficif workds in a column like:
```py
df[df['column'] == 'value']
```
or a a more general query looks like the following:
```py
def query_materials(request:str):
    if request == "": 
        return {}
    df = get_material_db()
    mask = df['Name (en)'].str.contains(request, case = False)
    data = df[mask]
    if data is not None:
        return data.to_json(orient='records')
    else:
        return {}
```

In [10]:
material_df[material_df['L4'] == 'Wood']

,Name (en),Name (de),Reference-size,Reference-unit,Density (kg/m3),Area weight (kg/m2),GWP,L1,L2,L3,L4,id
529,Window frame (spruce),Holz-Blendrahmen,1.0,m,NaN,NaN,"{'A1-A3': 0.0725391574, 'C1': 0.0, 'C2': 0.005...",Components for windows and curtain walls,Frames,Profiles,Wood,529
533,Window sash (spruce),Holz-Flügelrahmen,1.0,m,NaN,NaN,"{'A1-A3': 0.3177308448, 'C1': 0.0, 'C2': 0.005...",Components for windows and curtain walls,Frames,Profiles,Wood,533


This are the most general categories in the OBD

In [20]:
list(set(material_df['L1']))

['Others',
 'Building service engineering',
 'Coverings',
 'End of Life',
 'Insulation materials',
 'Wood',
 'Mineral building products',
 'Components for windows and curtain walls',
 'Plastics',
 'Metals']

## Selecting materials by category
This is how underspecifying a material might work

From the most general categories ``L1``, we filter ``'Plastics'`` and we can see the next level of categories available: `L2`

In [18]:
set(material_df[material_df['L1'] == 'Plastics']['L2'])

{'Elastic plastic profiles',
 'Floorings',
 'Foils and fleeces',
 'Profiles',
 'Roofing membranes',
 'Sealing materials',
 'Tubes'}

For example, if we want to underspecify a brick wall we might apply this filter, so that in each MC trial, the material can be *any* of the following materials:

In [22]:
material_df[material_df['L2'] == 'Bricks, blocks and elements'].head(5)

,Name (en),Name (de),Reference-size,Reference-unit,Density (kg/m3),Area weight (kg/m2),GWP,L1,L2,L3,L4,id
10,Adobe,Lehmstein,1.0,m3,1200.0,NaN,"{'A1-A3': 95.7593008607, 'C2': 3.719390796, 'C...",Mineral building products,"Bricks, blocks and elements",Air-dried brick (adobe),None,10
11,Aerated concrete P2 04 non-reinforced,Porenbeton P2 04 unbewehrt,1.0,m3,380.0,NaN,"{'A1-A3': 187.2448929873, 'C1': 0.1122719396, ...",Mineral building products,"Bricks, blocks and elements",Aerated concrete,None,11
12,Aerated concrete P4 05 non-reinforced,Porenbeton P4 05 unbewehrt,1.0,m3,472.0,NaN,"{'A1-A3': 227.4817141696, 'C1': 0.1394535671, ...",Mineral building products,"Bricks, blocks and elements",Aerated concrete,None,12
13,Aerated concrete P4 05 reinforced,Porenbeton P4 05 bewehrt,1.0,m3,500.0,NaN,"{'A1-A3': 262.0610543656, 'C1': 0.1559989055, ...",Mineral building products,"Bricks, blocks and elements",Aerated concrete,None,13
14,Aerated concrete granulate,Porenbeton Granulat,1.0,m3,400.0,NaN,"{'A1-A3': 5.2517004212, 'C1': 0.118180989, 'C2...",Mineral building products,"Bricks, blocks and elements",Aerated concrete,None,14


## Early version of calculating the impact of an Assembly

the structure is a dict so that
```py
{
    id_of_material: quantity(int), 
    id_of_material2: quantity2(int),
}
```


In [14]:
from inventory import assembly, ASSEMBLY_ATTRIBUTES


wall_str = {210: 0.1, 145: 0.5 }
generic_wall = assembly('exterior walls', 'masonry wall', 'bricks with 2 cm mortar', wall_str)


In [15]:
generic_wall.impact(material_df)

{'A1-A3': 59.63128623299001,
 'C1': 0.05909049452000001,
 'C2': 0.56412609097,
 'C3': 1.38014738565,
 'D': -0.31616394438,
 'C4': 0.195876173}

In [16]:
mask = material_df['Name (en)'].str.contains("adob", case=False)
material_df[mask]
# material_df.query('`Name (en)` == "Adob"')
# material_df.query('``Name (en)`` == @materialq')

,Name (en),Name (de),Reference-size,Reference-unit,Density (kg/m3),Area weight (kg/m2),GWP,L1,L2,L3,L4,id
10,Adobe,Lehmstein,1.0,m3,1200.0,NaN,"{'A1-A3': 95.7593008607, 'C2': 3.719390796, 'C...",Mineral building products,"Bricks, blocks and elements",Air-dried brick (adobe),None,10
